In [ ]:
from model_train import *
from model_evaluation import *
from graphs import *
from preprocess_data import *
from helpers import *
from clf_knn import KNN
from clf_decision_tree import DT

#### MODEL TRAINING ######
# set seed for cross-validation sampling
seed = 0

# set scoring function
scorer = make_scorer(balanced_f1)

# get dataset
data = get_abspath('seismic-bumps.csv', 'data/experiments')
df = pd.read_csv(data)

# set classifier params
data_name = 'seismic-bumps'
clf_name = 'DT'
knn = KNN()
dt = DT()

# get training and test splits
X_train, X_test, y_train, y_test = split_data(df, seed=seed)

# run grid search
grid = train_model(X_train, y_train, clf=dt, scorer=scorer, cv=5)

# save grid search results
save_train_results(grid, data_name, clf_name)

In [25]:
from model_train import *

# set seed for cross-validation sampling
seed = 0

# set scoring function
scorer = make_scorer(balanced_accuracy)

# load datasets
p_wine = get_abspath('winequality.csv', 'data/experiments')
p_seismic = get_abspath('seismic-bumps.csv', 'data/experiments')
df_wine = pd.read_csv(p_wine)
df_seismic = pd.read_csv(p_seismic)
dfs = {'wine': df_wine, 'seismic': df_seismic}
names = ['wine', 'seismic']

# instantiate classifiers
classifiers = {'KNN': KNN, 'DT': DT}

# begin training loop
for name in names:
    data_name = name
    X_train, X_test, y_train, y_test = split_data(dfs[name], seed=seed)

for key, value in classifiers.iteritems():
    clf_name = key
    clf = value()
    print clf.params
    
    

{'KNN__metric': ['manhattan', 'euclidean', 'chebyshev'], 'KNN__weights': ['uniform', 'distance'], 'KNN__n_neighbors': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33,
       35, 37, 39, 41, 43, 45, 47, 49])}
{'DT__criterion': ['gini', 'entropy'], 'DT__max_depth': array([ 1,  4,  7, 10, 13, 16, 19]), 'DT__class_weight': ['balanced'], 'DT__n_estimators': [100, 200, 500, 1000], 'DT__min_samples_leaf': array([1, 2, 3, 4])}


In [14]:
for datasets in 

      fixed_acidity  volatile_acidity  citric_acid  residual_sugar  chlorides  \
0               7.4             0.700         0.00            1.90      0.076   
1               7.8             0.880         0.00            2.60      0.098   
2               7.8             0.760         0.04            2.30      0.092   
3              11.2             0.280         0.56            1.90      0.075   
4               7.4             0.700         0.00            1.90      0.076   
5               7.4             0.660         0.00            1.80      0.075   
6               7.9             0.600         0.06            1.60      0.069   
7               7.3             0.650         0.00            1.20      0.065   
8               7.8             0.580         0.02            2.00      0.073   
9               7.5             0.500         0.36            6.10      0.071   
10              6.7             0.580         0.08            1.80      0.097   
11              7.5         